In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
data ={'Temp':[73,91,83,95,77,101],'Rainfall':[67,50,104,140,93,79],'Humidity':[45,35,78,98,56,43],'Apple':[50,40,65,100,75,55]}
data = pd.DataFrame(data)
data

,Temp,Rainfall,Humidity,Apple
0,73,67,45,50
1,91,50,35,40
2,83,104,78,65
3,95,140,98,100
4,77,93,56,75
5,101,79,43,55


In [3]:
inputs = data.drop('Apple',axis=1).values
inputs = inputs.astype('float32')
inputs

array([[ 73.,  67.,  45.],
       [ 91.,  50.,  35.],
       [ 83., 104.,  78.],
       [ 95., 140.,  98.],
       [ 77.,  93.,  56.],
       [101.,  79.,  43.]], dtype=float32)

In [4]:
target = data.Apple.values
target = target.astype('float32')
target

array([ 50.,  40.,  65., 100.,  75.,  55.], dtype=float32)

###### Converting inputs and target array to tensor

In [5]:
inputs = torch.from_numpy(inputs)
target = torch.from_numpy(target)
target = target.reshape(-1,1)

In [6]:
inputs , target

(tensor([[ 73.,  67.,  45.],
         [ 91.,  50.,  35.],
         [ 83., 104.,  78.],
         [ 95., 140.,  98.],
         [ 77.,  93.,  56.],
         [101.,  79.,  43.]]),
 tensor([[ 50.],
         [ 40.],
         [ 65.],
         [100.],
         [ 75.],
         [ 55.]]))

###### Linear Regression Equation
y = mx + c

m ---> weights/slope

c ---> constant

Equation for this problem will be like 

y(apple) = m1.x1(temp) + m2.x2(rainfall) + m3.x3(humidity) + c

In [7]:
# random weights and biases 
m = torch.randn(1, 3, requires_grad=True)
c = torch.randn(1, requires_grad=True)
print(m)
print(c)

tensor([[-0.7967,  2.8437,  0.5947]], requires_grad=True)
tensor([-1.4590], requires_grad=True)


In [8]:
# linear Regression model
def model(x):
    return x @ m.t() + c

In [9]:
prediction = model(inputs)
prediction

tensor([[157.6713],
        [ 89.0424],
        [274.5446],
        [379.2502],
        [234.9614],
        [168.2996]], grad_fn=<AddBackward0>)

In [10]:
target

tensor([[ 50.],
        [ 40.],
        [ 65.],
        [100.],
        [ 75.],
        [ 55.]])

In [11]:
#  loss function
def mse(y_true, prediction):
    diff = y_true - prediction
    diff_sq = diff * diff
    return torch.sum(diff_sq) / diff.numel()# number of elements

In [12]:
losses = mse(target,prediction)
losses

tensor(29052.0527, grad_fn=<DivBackward0>)

###### Back Propogation

In [13]:
losses.backward()

In [14]:
print(m) # randomly initialized weights
print(m.grad) # after backpropogation

tensor([[-0.7967,  2.8437,  0.5947]], requires_grad=True)
tensor([[26668.0391, 31460.2812, 21367.4688]])


###### Steps
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [15]:
preds = model(inputs)
print(preds)

tensor([[157.6713],
        [ 89.0424],
        [274.5446],
        [379.2502],
        [234.9614],
        [168.2996]], grad_fn=<AddBackward0>)


In [16]:
loss = mse(preds, target)
print(loss)

tensor(29052.0527, grad_fn=<DivBackward0>)


In [17]:
loss.backward()
print(m.grad)
print(c.grad)

tensor([[53336.0781, 62920.5625, 42734.9375]])
tensor([612.5130])


In [18]:
# Adjust weights & reset gradients
with torch.no_grad():
    m -= m.grad * 1e-5
    c -= c.grad * 1e-5
    m.grad.zero_()
    c.grad.zero_()

In [19]:
m

tensor([[-1.3300,  2.2145,  0.1673]], requires_grad=True)

In [20]:
c

tensor([-1.4651], requires_grad=True)

In [21]:
preds = model(inputs)
loss = mse(preds, target)
print(loss) # loss is reduced 

tensor(2951.7083, grad_fn=<DivBackward0>)


##### Updating the weights and biases for number of times

In [22]:
w = torch.randn(1, 3, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(w)
print(b)

tensor([[-1.9193,  0.6534, -1.2502]], requires_grad=True)
tensor([0.7610], requires_grad=True)


In [23]:
inputs , target

(tensor([[ 73.,  67.,  45.],
         [ 91.,  50.,  35.],
         [ 83., 104.,  78.],
         [ 95., 140.,  98.],
         [ 77.,  93.,  56.],
         [101.,  79.,  43.]]),
 tensor([[ 50.],
         [ 40.],
         [ 65.],
         [100.],
         [ 75.],
         [ 55.]]))

In [24]:
def model(x):
    return x @ w.t() + b
def mse(y_true, prediction):
    diff = y_true - prediction
    diff_sq = diff * diff
    return torch.sum(diff_sq) / diff.numel()# number of elements

In [25]:
for i in range(10000):
    pred = model(inputs)
    error=mse(pred, target)
    error.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [26]:
pred = model(inputs)
loss = mse(pred, target)
print(loss)

tensor(21.3631, grad_fn=<DivBackward0>)


In [27]:
pred , target

(tensor([[48.2370],
         [35.4390],
         [70.5378],
         [98.0977],
         [69.8209],
         [61.5696]], grad_fn=<AddBackward0>),
 tensor([[ 50.],
         [ 40.],
         [ 65.],
         [100.],
         [ 75.],
         [ 55.]]))

This process of updating the weights and biases is called as back propogation